In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 13.4MB/s 
     |████████████████████████████████| 3.3MB 51.8MB/s 
     |████████████████████████████████| 901kB 31.5MB/s 


In [48]:
import tensorflow as tf
from transformers import BertTokenizer, BertModel
import numpy as np
from typing import Callable, List, Optional, Tuple
import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import torch

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
from google.colab import drive

In [8]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
data = pd.read_csv("/content/gdrive/MyDrive/train.tsv", sep='\t') 

In [10]:
data

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [11]:
data = data.drop_duplicates(subset=['SentenceId'], keep='first')

In [12]:
data

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


In [13]:
bert_model = BertModel.from_pretrained("bert-base-uncased")

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [15]:
bert_model.cuda();

In [16]:
bert_model.device

device(type='cuda', index=0)

In [17]:
class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.tokenizer = bert_tokenizer
        self.model = bert_model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()
            #self.embedding_func = lambda x: torch.flatten(x[0])

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(text,
                                                    text,
                                                    add_special_tokens=True,
                                                    max_length=self.max_length,
                                                    truncation=True,
                                                    )["input_ids"]

        # Create an attention mask telling BERT to use all words
        attention_mask = [1] * len(tokenized_text)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(tokenized_text).unsqueeze(0).cuda(),
            torch.tensor(attention_mask).unsqueeze(0).cuda(),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings).detach().cpu()

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self

In [18]:
split = np.random.choice(
    ["train", "test"],
    size=data.shape[0],
    p=[.8, .2]
)

In [19]:
data["split"] = split

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
x_train = data[data["split"] == "train"]
y_train = x_train["Sentiment"]

In [21]:
x_train['Phrase'].apply(lambda x: x.split()).apply(len).max()

52

In [73]:
bert_transformer = BertTransformer(tokenizer, bert_model, max_length=60)
classifier = svm.SVC(C=1.0, class_weight="balanced", verbose=2)
scaler = StandardScaler()

In [72]:
tuned_parameters = param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['rbf'], average='micro'}
clf = GridSearchCV(svm.SVC(), param_grid = param_grid, cv=5, verbose=2, scoring= ['accuracy','f1', 'precision', 'recall'], n_jobs = -1, refit = 'accuracy')

SyntaxError: ignored

In [67]:
bert_transformer = BertTransformer(tokenizer, bert_model, max_length=60)
classifier = clf

In [74]:
model = Pipeline(steps=
    [
        ("vectorizer", bert_transformer),
        ('scaler', StandardScaler()),
        ("classifier", classifier),
    ], verbose = True
)

In [ ]:
model.fit(x_train["Phrase"], y_train)

[Pipeline] ........ (step 1 of 3) Processing vectorizer, total= 1.1min
[Pipeline] ............ (step 2 of 3) Processing scaler, total=   0.1s
[LibSVM][Pipeline] ........ (step 3 of 3) Processing classifier, total= 1.0min


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7f488be14cb0>,
                                 max_length=60)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=2))],
         verbose=True)

In [26]:
x_test = data[data["split"] == "test"]
y_test = x_test["Sentiment"]

Result with one input

In [36]:
 model.score(x_test['Phrase'], y_test)

0.4044495151169424

Result with two inputs

In [75]:
model.fit(x_train["Phrase"], y_train)

[Pipeline] ........ (step 1 of 3) Processing vectorizer, total= 1.2min
[Pipeline] ............ (step 2 of 3) Processing scaler, total=   0.1s
[LibSVM][Pipeline] ........ (step 3 of 3) Processing classifier, total= 1.1min


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7f4f1aba9dd0>,
                                 max_length=60)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=2))],
         verbose=True)

In [76]:
y_pred=model.predict(x_train["Phrase"])

In [77]:
 model.score(x_test['Phrase'], y_test)

0.4409583571021107

In [ ]:
model.cv_results_['mean_test_accuracy'][0]

In [78]:
from sklearn.metrics import classification_report
sentiment = ['class 0', 'class 1', 'class 2','class3', 'class4']
print(classification_report(y_train, y_pred, target_names=sentiment))

              precision    recall  f1-score   support

     class 0       0.72      0.93      0.81       866
     class 1       0.82      0.79      0.81      1751
     class 2       0.80      0.78      0.79      1314
      class3       0.87      0.72      0.79      1847
      class4       0.77      0.89      0.83       998

    accuracy                           0.80      6776
   macro avg       0.80      0.82      0.80      6776
weighted avg       0.81      0.80      0.80      6776



In [79]:
y_test_pred=model.predict(x_test['Phrase'])

In [80]:
sentiment = ['class 0', 'class 1', 'class 2','class3', 'class4']
print(classification_report(y_test, y_test_pred, target_names=sentiment))

              precision    recall  f1-score   support

     class 0       0.36      0.46      0.40       206
     class 1       0.46      0.44      0.45       449
     class 2       0.35      0.35      0.35       341
      class3       0.49      0.41      0.44       474
      class4       0.53      0.59      0.56       283

    accuracy                           0.44      1753
   macro avg       0.44      0.45      0.44      1753
weighted avg       0.44      0.44      0.44      1753



gridsearchcv

In [58]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = GridSearchCV(
      SVC(), tuned_parameters, scoring='%s_macro' % score
  )
 

In [ ]:
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)

with two outputs

In [30]:
model.fit(x_train["Phrase"], y_train)

[Pipeline] ........ (step 1 of 3) Processing vectorizer, total= 1.2min
[Pipeline] ............ (step 2 of 3) Processing scaler, total=   0.1s
[LibSVM][Pipeline] ........ (step 3 of 3) Processing classifier, total= 1.1min


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7f71bbfe1b00>,
                                 max_length=60)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=2))],
         verbose=True)

In [31]:
 model.score(x_test['Phrase'], y_test)

0.4192601291837933

Data Augmentation

In [ ]:
!pip install textaugment

     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 1.0MB 12.7MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp37-none-any.whl size=15737 sha256=0874668a8db5c9b28a791577909c773e8ae44fc0237696ccc3b7ab92e3385a81
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
Successfully built googletrans


In [ ]:
from textaugment import Word2vec
import gensim
import gdown

Word2vec embedding

In [ ]:
url ='https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM'
output = 'GoogleNews-vectors-negative300.bin.gz'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:09, 180MB/s]


'GoogleNews-vectors-negative300.bin.gz'

In [ ]:
!gzip -d /content/GoogleNews-vectors-negative300.bin.gz

In [ ]:
genModel = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True, limit=10000)

In [ ]:
t = Word2vec(model=genModel)
t.augment('The stories are good')

'another headlines were good'

In [ ]:
runs = 1 # By default.
v = False # verbose mode to replace all the words. If enabled runs is not effective. Used in this paper (https://www.cs.cmu.edu/~diyiy/docs/emnlp_wang_2015.pdf)
p = 0.5 # The probability of success of an individual trial. (0.1<p<1.0), default is 0.5. Used by Geometric distribution to selects words from a sentence.
t = Word2vec(model=genModel, runs=5, v=False, p=0.5)
t.augment('The stories are good')

'another story was great'

In [ ]:
aug_data = []
aug_sentiment = []
for index, row in data.iterrows():
  aug_data.append(t.augment(row['Phrase']))
  aug_sentiment.append(row['Sentiment'])

In [ ]:
new_data = {'Phrase':aug_data, 'Sentiment':aug_sentiment}
newData_df = data.append(pd.DataFrame(new_data))

In [ ]:
newData_df.shape[0]

17058

In [ ]:
split = np.random.choice(
    ["train", "test"],
    size=newData_df.shape[0],
    p=[.8, .2]
)

In [ ]:
newData_df["split"] = split

In [ ]:
x_train = newData_df[newData_df["split"] == "train"]
y_train = x_train["Sentiment"]

In [ ]:
x_train['Phrase'].apply(lambda x: x.split()).apply(len).max()

52

In [ ]:
model.fit(x_train['Phrase'], y_train)

[Pipeline] ........ (step 1 of 3) Processing vectorizer, total= 1.9min
[Pipeline] ............ (step 2 of 3) Processing scaler, total=   0.2s
[LibSVM][Pipeline] ........ (step 3 of 3) Processing classifier, total= 4.4min


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7f42570cfa70>,
                                 max_length=60)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=2))],
         verbose=True)

In [ ]:
x_test = newData_df[newData_df["split"] == "test"]
y_test = x_test["Sentiment"]

In [ ]:
model.score(x_test['Phrase'], y_test)

0.4273528575490475

Data Augmentation using Pegasus transformer

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 12.3MB/s 


In [ ]:
import torch
import sentencepiece
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
Pegmodel = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Data Augmentation using Bert

In [ ]:
from transformers import pipeline

In [ ]:
def create_pipeline(model_name):
   return pipeline('fill-mask', model=model_name)

In [ ]:
predict_mask = create_pipeline("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
predict_mask("men should wear shirt and [MASK] for tomorrow's event.")

[{'score': 0.3029894232749939,
  'sequence': "men should wear shirt and tie for tomorrow's event.",
  'token': 5495,
  'token_str': 'tie'},
 {'score': 0.2420877367258072,
  'sequence': "men should wear shirt and pants for tomorrow's event.",
  'token': 6471,
  'token_str': 'pants'},
 {'score': 0.18802925944328308,
  'sequence': "men should wear shirt and jeans for tomorrow's event.",
  'token': 6312,
  'token_str': 'jeans'},
 {'score': 0.08707275241613388,
  'sequence': "men should wear shirt and trousers for tomorrow's event.",
  'token': 15292,
  'token_str': 'trousers'},
 {'score': 0.07159349322319031,
  'sequence': "men should wear shirt and shorts for tomorrow's event.",
  'token': 9132,
  'token_str': 'shorts'}]

In [ ]:
len(a)

5

In [ ]:
def random_mask(text):
  text = text.split()
  rand_idx = np.random.randint(0, len(text))
  text[rand_idx] = '[MASK]'
  return ' '.join(text)

In [ ]:
#data_lens = data['Phrase'].apply(len).describe()
#data = data.loc[:, :, data_lens >= 10, :]
data = data[data['Phrase'].apply(len)>10]

In [ ]:
masked = []
masked_sentiment = []
for index, row in data.iterrows():
  a=''
  a=predict_mask(random_mask(row['Phrase']))
  masked.append(a[0]['sequence'])
  masked_sentiment.append(row['Sentiment'])

In [ ]:
new_bdata = {'Phrase':masked, 'Sentiment':masked_sentiment}
newbData_df = data.append(pd.DataFrame(new_bdata))

In [ ]:
split = np.random.choice(
    ["train", "test"],
    size=newbData_df.shape[0],
    p=[.8, .2]
)

In [ ]:
newbData_df["split"] = split

In [ ]:
x_train = newbData_df[newbData_df["split"] == "train"]
y_train = x_train["Sentiment"]

In [ ]:
x_train['Phrase'].apply(lambda x: x.split()).apply(len).max()

57

In [ ]:
model.fit(x_train['Phrase'], y_train)

[Pipeline] ........ (step 1 of 3) Processing vectorizer, total= 2.2min
[Pipeline] ............ (step 2 of 3) Processing scaler, total=   0.2s
[LibSVM][Pipeline] ........ (step 3 of 3) Processing classifier, total= 4.0min


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7f7ecc8bb7a0>,
                                 max_length=60)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=2))],
         verbose=True)

In [ ]:
x_test = newbData_df[newbData_df["split"] == "test"]
y_test = x_test["Sentiment"]

In [ ]:
model.score(x_test['Phrase'], y_test)

0.6724852071005917